In [11]:
import sqlite3

conn = sqlite3.connect('orgdb.sqlite')
cur = conn.cursor()

cur.execute('DROP TABLE IF EXISTS Counts')

cur.execute('''
CREATE TABLE Counts (org TEXT, count INTEGER)''')

fname = input('Enter file name: ')
if (len(fname) < 1): fname = 'src/mbox-short.txt'
fh = open(fname)
for line in fh:
    if not line.startswith('From '): continue
    email = line.split()[1]
    org = email.split('@')[1]
    cur.execute('SELECT count FROM Counts WHERE org = ? ', (org,))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count)
                VALUES (?, 1)''', (org,))
    else:
        cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?',
                    (org,))
    conn.commit()

# https://www.sqlite.org/lang_select.html
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'

for row in cur.execute(sqlstr):
    print(str(row[0]), row[1])

cur.close()

iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17


In [14]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
DROP TABLE IF EXISTS Genre;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')


fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'src/tracks/Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname)
all = stuff.findall('dict/dict/dict')
print('Dict count:', len(all))
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')
    genre = lookup(entry, 'Genre')

    if name is None or artist is None or album is None or genre_id is None: 
        continue

    print(name, artist, album, count, rating, length, genre)

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    
    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ? )''', ( genre, ) )
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, genre_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ?, ? )''', 
        ( name, album_id, genre_id, length, rating, count ) )

    conn.commit()


Dict count: 404
Another One Bites The Dust Queen Greatest Hits 55 100 217103 Rock
Asche Zu Asche Rammstein Herzeleid 79 100 231810 Industrial
Beauty School Dropout Various Grease 48 100 239960 Soundtrack
Black Dog Led Zeppelin IV 109 100 296620 Rock
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] 33 100 87118 Rock
Circles Bryan Lee Blues Is 54 60 355369 Funk
Comfortably Numb Pink Floyd The Wall [Disc 2] 36 100 384130 Rock
Crazy Little Thing Called Love Queen Greatest Hits 38 100 163631 Rock
Electric Funeral Black Sabbath Paranoid 44 100 293015 Metal
Fat Bottomed Girls Queen Greatest Hits 38 100 257515 Rock
For Those About To Rock (We Salute You) AC/DC Who Made Who 84 100 353750 Rock
Four Sticks Led Zeppelin IV 84 100 284421 Rock
Furious Angels Rob Dougan The Matrix Reloaded 54 100 330004 Soundtrack
Gelle Bryan Lee Blues Is 45 60 199836 Blues/R&B
Going To California Led Zeppelin IV 100 100 215666 Rock
Grease Various Grease 42 100 205792 Soundtrack
Hand of Doom Black Sabbath Parano

TypeError: 'NoneType' object is not subscriptable

In [23]:
import json
import sqlite3

conn = sqlite3.connect('rosterdb.sqlite')
cur = conn.cursor()

# Do some setup
cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;

CREATE TABLE User (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT UNIQUE
);

CREATE TABLE Course (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title  TEXT UNIQUE
);

CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id)
)
''')

fname = input('Enter file name: ')
if len(fname) < 1:
    fname = 'roster_data.json'

# [
#   [ "Charley", "si110", 1 ],
#   [ "Mea", "si110", 0 ],

str_data = open(fname).read()
json_data = json.loads(str_data)

for entry in json_data:

    name = entry[0]
    title = entry[1]
    role = entry[2]

    print((name, title, role))

    cur.execute('''INSERT OR IGNORE INTO User (name)
        VALUES ( ? )''', ( name, ) )
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Course (title)
        VALUES ( ? )''', ( title, ) )
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id, role) VALUES ( ?, ?, ? )''',
        ( user_id, course_id, role ) )
    
    cur.execute('SELECT role FROM Member WHERE user_id = ? AND course_id = ?', (user_id, course_id))
    role = cur.fetchone()[0]

    conn.commit()

('Riva', 'si110', 1)
('Rikki', 'si110', 0)
('Chenille', 'si110', 0)
('Bilal', 'si110', 0)
('Codey', 'si110', 0)
('Nick', 'si110', 0)
('Amos', 'si110', 0)
('Rheyden', 'si110', 0)
('Blessing', 'si110', 0)
('Avya', 'si110', 0)
('Jole', 'si110', 0)
('Esme', 'si110', 0)
('Emmanuel', 'si110', 0)
('Mohaddesa', 'si110', 0)
('Ketsia', 'si110', 0)
('Lillie', 'si110', 0)
('Sethu', 'si110', 0)
('LLeyton', 'si110', 0)
('Morvyn', 'si110', 0)
('Havin', 'si110', 0)
('Lia', 'si110', 0)
('Connar', 'si110', 0)
('Kiran', 'si110', 0)
('Kayleigh', 'si110', 0)
('Nikash', 'si110', 0)
('Lance', 'si110', 0)
('Kenan', 'si110', 0)
('Sanaa', 'si110', 0)
('Archibald', 'si110', 0)
('Cailyn', 'si110', 0)
('Kennedy', 'si110', 0)
('Bronagh', 'si110', 0)
('Zen', 'si110', 0)
('Benjamin', 'si110', 0)
('Ennis', 'si106', 1)
('Raul', 'si106', 0)
('Rasul', 'si106', 0)
('Name', 'si106', 0)
('Xanthe', 'si106', 0)
('Vasemaca', 'si106', 0)
('Cathal', 'si106', 0)
('Fern', 'si106', 0)
('Daniyal', 'si106', 0)
('Abid', 'si106', 0)
('